# STINTSY S14 MP

### Group Members:
- ALCANTARA, KEIRA GABRIELLE CAFE
- DE JESUS, IAN GABRIEL SANDOVAL
- DIWAJON, HECTOR ZACHARY CALLANGA
- RESURRECCION, ALJIRAH BRENDL YUTAN

# Introduction

# Dataset Columns and Descriptions

| Column Name        | Description                                                  |
|--------------------|--------------------------------------------------------------|
| W_REGN             | Regions ranging from 1 to 16 (only exception is 4 having the numbers 41 for Region IVa and 42 for Region IVb)|
| W_OID              | Other ID's for each province arranged in increments of one alphabetically up to the last. (Abra being 1, Agusan Del Norte 2 and so on)|
| W_SHSN             | Sample household serial number. |

# Imported Libraries: 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read Dataset:

In [4]:
ds_income_df = pd.read_csv("FIES PUF 2012 Vol.1.CSV")
ds_income_df

,W_REGN,W_OID,W_SHSN,W_HCN,URB,RSTR,PSU,BWEIGHT,RFACT,FSIZE,...,PC_QTY,OVEN_QTY,MOTOR_BANCA_QTY,MOTORCYCLE_QTY,POP_ADJ,PCINC,NATPC,NATDC,REGDC,REGPC
0,14,101001000,2,25,2,21100,415052,138.25,200.6576,3.0,...,01,01,,,0.946172,108417.00,9,8,8,9
1,14,101001000,3,43,2,21100,415052,138.25,200.6576,12.5,...,,01,,01,0.946172,30631.60,5,9,9,4
2,14,101001000,4,62,2,21100,415052,138.25,200.6576,2.0,...,,01,,,0.946172,86992.50,9,6,6,8
3,14,101001000,5,79,2,21100,415052,138.25,200.6576,4.0,...,,01,,,0.946172,43325.75,6,6,6,6
4,14,101001000,10,165,2,21100,415052,138.25,200.6576,5.0,...,,,,01,0.946172,37481.80,6,6,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40166,12,9804029001,18,568,1,22000,114062,271.25,963.2264,5.0,...,00,00,00,00,0.902863,30101.60,5,5,7,7
40167,12,9804035000,1,25,2,22000,414067,271.25,588.6253,9.0,...,00,01,00,00,0.902863,14368.89,1,5,7,3
40168,12,9804035000,2,51,2,22000,414067,271.25,588.6253,6.0,...,00,00,00,00,0.902863,19137.33,3,4,6,4
40169,12,9804035000,3,75,2,22000,414067,271.25,588.6253,5.0,...,00,01,00,00,0.902863,30985.00,5,6,7,7


In [11]:
ds_income_df.info()

observation_count, variables_count = ds_income_df.shape
print("\nNumber of observations:", observation_count)
print("Number of variables:", variables_count)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40171 entries, 0 to 40170
Columns: 119 entries, W_REGN to REGPC
dtypes: float64(5), int64(92), object(22)
memory usage: 36.5+ MB

Number of observations: 40171
Number of variables: 119


In [13]:
# check for missing values
ds_missing_val = ds_income_df.isnull().sum()
print("Columns with missing values: ", ds_missing_val[ds_missing_val > 0])

Columns with missing values:  Series([], dtype: int64)


In [ ]:
# checking of unique values

for column in ds_income_df.columns:
    unique_values = ds_income_df[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'W_REGN': [14 16  6  5 15  3  2 41  7 10 11  8  1 13 42 12  9]
Unique values in column 'W_OID': [ 101001000  101002000  101023000 ... 9804028001 9804029001 9804035000]
Unique values in column 'W_SHSN': [ 2  3  4  5 10 14 16 18 20 21 22 24  1  6  7  8  9 11 12 13 17 19 15 23
 25 26 27 28 29 30]
Unique values in column 'W_HCN': [  25   43   62   79  165  229  262  295  329  344  362  392   27   44
   89  113  140  162  185  209  233  256  281  302  325  418    3    1
   17  121  139  157  174  191  222  255  273  307  324  340  358   61
 8001 8002   77  160  177  226  339  378   13   21   28   36   45   64
   87   96  105  114  130  146  154  171   20   76   90  104  118  133
  148  161  188   11   32   52   72   83   93  115  125  135  143  151
  173  216  227    4   56   68  111  123  156  166  189  200  219  228
   10   30   41   59   78  107  117  178    5    7   19  102  124  136
  195    9   24   40   48   51   54   58   15   34   74   81   99  108
  127  15